# Medicaid/Medicare Drug Spending Data (2012-2020)

## Data Cleaning

In [1]:
# you did run this from the notebooks folder, right? :)
pwd = !pwd
assert pwd[0][-9:]=="notebooks"

In [2]:
#pandas for dataframes/reading in csv data/saving csv data
import pandas as pd

#numeric types/computation, vectorized pandas operations
import numpy as np

### Read in the data

In [3]:
filename = "../data/combined/spending.csv"
med = pd.read_csv(filename, header=0, index_col=0)
filename2 = "../data/combined/drug_info.csv"
drug_info = pd.read_csv(filename2, header=0, index_col=0)

### Check columns names

In [4]:
med.columns

Index(['Brand Name', 'Generic Name', 'Number of Manufacturers',
       'Total Spending', 'Total Dosage Units', 'Total Claims',
       'Average Spending Per Dosage Unit (Weighted)',
       'Average Spending Per Claim', 'year', 'coverage_type'],
      dtype='object')

### Replace spaces in column names for easier use and convert to lower case

In [5]:
med.columns = [col.replace(' ', '_').lower() for col in list(med.columns)]
med.columns

Index(['brand_name', 'generic_name', 'number_of_manufacturers',
       'total_spending', 'total_dosage_units', 'total_claims',
       'average_spending_per_dosage_unit_(weighted)',
       'average_spending_per_claim', 'year', 'coverage_type'],
      dtype='object')

## 1. Cleaning drug use info data




In [6]:
drug_info.columns

Index(['Brand', 'Generic', 'Uses'], dtype='object')

In [7]:
drug_info.columns = ["brand_name", "generic_name", "uses"]
drug_info.columns

Index(['brand_name', 'generic_name', 'uses'], dtype='object')

In [8]:
drug_info.head(10)

,brand_name,generic_name,uses
0,8hr arthritis pain,Acetaminophen,This drug is used to treat mild to moderate pa...
1,a & d,Vitamins A And D,This medication is used as a moisturizer to tr...
2,a & d diaper rash,"Dimethic/Zinc Ox/Vits A,D/Aloe",Drug uses not available
3,a-hydrocort,Hydrocortisone Sod Succinate,This medication is used to treat various condi...
4,a-methapred,Methylprednisolone Sod Succ,This medication is used to treat conditions su...
5,abacavir,Abacavir Sulfate,This drug is used with other HIV medications t...
6,abacavir-lamivudine,Abacavir Sulfate/Lamivudine,This product contains 2 drugs: abacavir and la...
7,abacavir-lamivudine-zidovudine,Abacavir/Lamivudine/Zidovudine,"This product contains 3 drugs: abacavir, lamiv..."
8,abdek multivitamin,Pediatric Multivit 152/D3/K,This medication is a multivitamin product used...
9,abelcet,Amphotericin B Lipid Complex,This medication is used to treat a variety of ...


#### Convert names to lowercase and strip "*" at beginning or end

In [9]:
drug_info['generic_name'] = drug_info.generic_name.str.lower()
drug_info["brand_name"] = drug_info["brand_name"].apply(lambda x: x.strip('*'))

In [10]:
drug_info.shape

(4593, 3)

In [11]:
drug_info.drop_duplicates(inplace=True)

In [12]:
drug_info.shape

(4590, 3)

#### Dropped three duplicate rows of drugs. Then we'll drop rows without a drug use description.

In [13]:
drug_info.uses.value_counts().head(3)

Drug uses not available                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   724
This combination hormone medication is used to prevent pregnancy. It contains 2 hormones: a progestin and an estrogen. It works mainly by preventing the release of an egg (ovulation) during your menstru

In [14]:
drugs_with_use = drug_info[drug_info["uses"] != "Drug uses not available"].copy()
drugs_with_use.shape

(3866, 3)

#### We have left 88 hormone drugs for birth control with different names (or they'd have been dropped as duplicates). Additionally, there are lots of multivitamins in the data. Below, we show the number of unique uses.

In [15]:
drugs_with_use.uses.unique().shape

(2106,)

## 2. Cleaning the spending data

In [16]:
med.loc[:,"brand_name"] = med["brand_name"].str.lower()
med.loc[:,"generic_name"] = med["generic_name"].str.lower()

In [17]:
med.shape

(70810, 10)

In [18]:
med = med.dropna()

In [19]:
med.drop_duplicates(subset=['brand_name', 'generic_name', 'year', 'coverage_type'], inplace=True)

In [20]:
med.shape

(59573, 10)

In [21]:
med['coverage_type'] = med['coverage_type'].str.strip()
med['brand_name'] = med['brand_name'].str.strip()
med['generic_name'] = med['generic_name'].str.strip()


In [22]:
med.year.value_counts()

2016    9427
2020    7971
2019    7301
2018    6652
2015    6191
2017    6001
2014    5789
2013    5325
2012    4916
Name: year, dtype: int64

#### Note the abnormally high counts for 2016. We have some duplicates in the 2016 data which we will drop below

In [23]:
med.drop_duplicates(subset=['brand_name', 'generic_name', 'year', 'coverage_type'], inplace=True)

In [25]:
med.dtypes

brand_name                                      object
generic_name                                    object
number_of_manufacturers                          int64
total_spending                                 float64
total_dosage_units                             float64
total_claims                                   float64
average_spending_per_dosage_unit_(weighted)    float64
average_spending_per_claim                     float64
year                                             int64
coverage_type                                   object
dtype: object

In [26]:
drugs_with_use.head()

,brand_name,generic_name,uses
0,8hr arthritis pain,acetaminophen,This drug is used to treat mild to moderate pa...
1,a & d,vitamins a and d,This medication is used as a moisturizer to tr...
3,a-hydrocort,hydrocortisone sod succinate,This medication is used to treat various condi...
4,a-methapred,methylprednisolone sod succ,This medication is used to treat conditions su...
5,abacavir,abacavir sulfate,This drug is used with other HIV medications t...


### 3. Our data is looking much cleaner after dropping Null values and duplicates.

In [27]:
# with a few high value columns, lets save space by dropping the decimals
med.loc[:,["total_spending","total_dosage_units", "total_claims"]] = \
med.loc[:,["total_spending","total_dosage_units", "total_claims"]].astype(int)


In [28]:
med.head()

,brand_name,generic_name,number_of_manufacturers,total_spending,total_dosage_units,total_claims,average_spending_per_dosage_unit_(weighted),average_spending_per_claim,year,coverage_type
1,a & d,vitamins a and d,1,470,7992,49,0.06,9.61,2016,medicaid
5,a-methapred,methylprednisolone sod succ,1,34520,2426,1189,15.28,29.03,2016,medicaid
6,abacavir,abacavir sulfate,7,7360772,1613893,27596,4.56,266.73,2016,medicaid
7,abacavir-lamivudine,abacavir sulfate/lamivudine,6,9569547,283169,9466,33.79,1010.94,2016,medicaid
8,abacavir-lamivudine-zidovudine,abacavir/lamivudine/zidovudine,1,6253710,284214,4804,22.00,1301.77,2016,medicaid


#### One last missing value check:

In [29]:
med.isna().sum()

brand_name                                     0
generic_name                                   0
number_of_manufacturers                        0
total_spending                                 0
total_dosage_units                             0
total_claims                                   0
average_spending_per_dosage_unit_(weighted)    0
average_spending_per_claim                     0
year                                           0
coverage_type                                  0
dtype: int64

In [30]:
drugs_with_use.isna().sum()

brand_name      0
generic_name    0
uses            0
dtype: int64

In [31]:
drugs_with_use.to_csv("../data/clean/drug_uses_clean.csv")
med.to_csv("../data/clean/spending_cleaned.csv")